In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import datetime
print(f"last edit: {datetime.datetime.now()}")

last edit: 2025-07-26 08:21:00.036859


In [11]:
!nvidia-smi

Sat Jul 26 08:21:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
# Get data (10% of labels)
import zipfile

# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2025-07-26 08:21:00--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.207, 74.125.142.207, 74.125.195.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   207MB/s    in 0.8s    

2025-07-26 08:21:01 (207 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [13]:
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"there are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}")

there are 2 directories and 0 images in '10_food_classes_10_percent
there are 10 directories and 0 images in '10_food_classes_10_percent/test
there are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream
there are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi
there are 0 directories and 250 images in '10_food_classes_10_percent/test/steak
there are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice
there are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza
there are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry
there are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings
there are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen
there are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger
there are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon
there are 

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size = IMAGE_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode ="categorical")

print("Testing images:")

test_data_10_percent = test_datagen.flow_from_directory(test_dir,
                                                          target_size = IMAGE_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode ="categorical")

Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


##**Setting up callback**

In [16]:
# Create tensorboard callback
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name+ "/" + experiment_name + "/" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.tensorboard(
      lpg_dir=log_dir
  )
  print(f"Saving tensorboard log files to:{log_dir}")
  return tensorboard_callback